In [1]:
from __future__ import annotations

%load_ext jupyter_black

In [2]:
import numpy as np
import mesoscaler as ms

This application uses Enums to define Independent (ms.Coordinates and ms.Dimensions) and Dependent Variables (Variables).
In order to process multiple datasets a standard Coordinate and Dimension naming convention is required. Much of the
scheme was taken from the Climate and Forecast Metadata Conventions (CF).

# Coordinates and Dimensions (Independent Variables)
The enums are a subset of a string. So their actual `.value` is a true `LiteralString` and the `EnumMember` is subset
of that. The `Enum.__call__` can either be called with the member.name, member.value, the member it'self or any of the aliases.


In [3]:
print(
    ms.Coordinates,
    ms.Dimensions,
    f"""\
{ms.Coordinates.vertical.name = }
{ms.Coordinates.vertical.value = }
{ms.Coordinates.vertical.axis = }
{ms.Coordinates.vertical.aliases = }
{ms.Coordinates(ms.Coordinates.vertical.aliases) = }
{ms.Coordinates('height') = }
{ms.Coordinates('level') = }
""",
    sep="\n\n",
)
assert ms.Coordinates("level") == ms.Coordinates.vertical == ms.Coordinates("height") == "vertical"
assert ms.Coordinates.vertical.axis == (ms.Dimensions.Z,)
assert all(issubclass(x, ms.IndependentVariables) for x in (ms.Coordinates, ms.Dimensions))

Coordinates:
-      time: time
-  vertical: vertical
-  latitude: latitude
- longitude: longitude

Dimensions:
- T: T
- Z: Z
- Y: Y
- X: X

ms.Coordinates.vertical.name = 'vertical'
ms.Coordinates.vertical.value = 'vertical'
ms.Coordinates.vertical.axis = (Z,)
ms.Coordinates.vertical.aliases = ['level', 'height']
ms.Coordinates(ms.Coordinates.vertical.aliases) = [vertical]
ms.Coordinates('height') = vertical
ms.Coordinates('level') = vertical



# ERA5 and URMA (Dependent Variables)

The 2 datasets used in used initially are the era5 and urma, there are some scripts do download and process the data
into `.zarr` files. 

In [4]:
print(ms.ERA5, ms.URMA, sep="\n")
assert ms.ERA5("Z") is ms.ERA5("z") is ms.ERA5.Z is ms.ERA5("geopotential") and ms.ERA5.Z == "geopotential"
assert ms.ERA5("z") is ms.ERA5.Z
assert ms.ERA5("z") == ms.ERA5.Z
assert (
    ms.ERA5("u") is ms.ERA5["U"] is ms.ERA5.loc["U"] is ms.ERA5.U is ms.ERA5("u_component_of_wind")
    and ms.ERA5.U == "u_component_of_wind"
)

assert (
    set(ms.ERA5).difference(ms.ERA5(["u", "v"]))
    == ms.ERA5.difference(["u", "v"])
    == {ms.ERA5.Q, ms.ERA5.T, ms.ERA5.W, ms.ERA5.Z}
)
assert set(ms.ERA5).intersection(ms.ERA5(["u", "v"])) == ms.ERA5.intersection(["u", "v"]) == {ms.ERA5.U, ms.ERA5.V}
assert all(issubclass(x, ms.DependentVariables) for x in (ms.ERA5, ms.ERA5))

ERA5:
- Z: geopotential
- Q: specific_humidity
- T: temperature
- U: u_component_of_wind
- V: v_component_of_wind
- W: vertical_velocity
URMA:
-    TCC: total_cloud_cover
-   CEIL: ceiling
-    U10: u_wind_component_10m
-    V10: v_wind_component_10m
-   SI10: wind_speed_10m
-   GUST: wind_speed_gust
- WDIR10: wind_direction_10m
-    T2M: temperature_2m
-    D2M: dewpoint_temperature_2m
-    SH2: specific_humidity_2m
-     SP: surface_pressure
-    VIS: visibility
-   OROG: orography


In [5]:
(u, v), z = ms.ERA5.loc[["U", "V"]], ms.ERA5.loc["Z"]

u, v, z

(u_component_of_wind, v_component_of_wind, geopotential)

In [6]:
ms.ERA5.intersection(["u", "v"])

{u_component_of_wind, v_component_of_wind}

In [7]:
# the crs is loaded lazily
assert "crs" not in ms.ERA5.metadata
print(repr(ms.ERA5.crs))
assert "crs" in ms.ERA5.metadata

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich



In [8]:
print(
    ms.ERA5.metadata,
    ms.ERA5.Z.metadata,
    ms.ERA5.__metadata__,
    sep="\n",
)

{'crs': <Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
}
{'units': 'm**2 s**-2'}
{'name': 'ERA5', '__metadata_cls_data__': {'crs': <Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
}, '__metadata_loc__': <mesoscaler._metadata._Loc object at 0x7f0cd945dcf0>, '__metadata_member_aliases__':    Z  Q  T  U  V  W
0  z  q  t  u  v  w, '__metadata_series__': member_names
Z           geopotential
Q      specific_humidity
T            temperature
U    u_component_of_wind
V    v_compon

In [9]:
print(
    ms.ERA5.difference(list("tuv")),
    ms.ERA5.difference(ms.ERA5(list("tuv"))),
    ms.ERA5(list("tuv")),
    sep="\n",
)

{specific_humidity, vertical_velocity, geopotential}
{specific_humidity, vertical_velocity, geopotential}
[temperature, u_component_of_wind, v_component_of_wind]


In [10]:
assert ms.Coordinates.intersection(["vertical", "time", "latitude", "longitude"]) == set(ms.Coordinates)
assert ms.Coordinates.difference(list(ms.Coordinates)) == set() == set(ms.Coordinates).difference(iter(ms.Coordinates))
assert ms.Dimensions.intersection(["time", "latitude", "longitude"]) == {
    ms.Dimensions.T,
    ms.Dimensions.X,
    ms.Dimensions.Y,
}

## time64.Time64

The time64 module provides an interface to `numpy.datetime64` and `numpy.timedelta64` objects and `Array[..., np.datetime64 | np.timedelta64]` 

with methods for creating time batches and time ranges.

In [11]:
import datetime

assert (
    "h"
    == ms.hours
    == ms.time64.Time64("h")
    == ms.time64.Time64("hours")
    == ms.time64.Time64("datetime64[h]")
    == ms.time64.Time64("timedelta64[h]")
)
assert (
    ms.hours.datetime("2022-01-01")
    == ms.hours.datetime(2022, 1, 1)
    == ms.hours.datetime(datetime.datetime(2022, 1, 1))
    == ms.hours.datetime(np.datetime64("2022-01-01"))
    == np.datetime64("2022-01-01", "h")
)
assert ms.hours.delta(2) == np.timedelta64(2, "h")
assert ms.hours.infer_dtype(datetime.datetime(2022, 1, 1)) == np.dtype("datetime64[h]")
assert ms.hours.infer_dtype(datetime.timedelta(hours=1)) == np.dtype("timedelta64[h]")


ms.hours.arange("2022-01-01", "2022-01-02", 2)


ms.hours.arange("2022-01-01", "2022-01-02", 2)

array(['2022-01-01T00', '2022-01-01T02', '2022-01-01T04', '2022-01-01T06',
       '2022-01-01T08', '2022-01-01T10', '2022-01-01T12', '2022-01-01T14',
       '2022-01-01T16', '2022-01-01T18', '2022-01-01T20', '2022-01-01T22'],
      dtype='datetime64[h]')

In [12]:
try:
    np.arange("2022-01-01", "2022-01-06", 6)
except TypeError as e:
    print('not supported: np.arange("2022-01-01", "2022-01-06", 6)')


try:
    ms.hours.batch("2022-01-01", "2023-01-01", 6, size=40)
except ValueError as e:
    print(e)
ms.hours.batch("2022-01-01", "2023-01-01", 6, size=20)

not supported: np.arange("2022-01-01", "2022-01-06", 6)
Array size 1460 is not divisible by batch size 40.
try using a size of:
[   1    2    4    5   10   20   73  146  292  365  730 1460]


array([['2022-01-01T00', '2022-01-01T06', '2022-01-01T12', ...,
        '2022-01-05T06', '2022-01-05T12', '2022-01-05T18'],
       ['2022-01-06T00', '2022-01-06T06', '2022-01-06T12', ...,
        '2022-01-10T06', '2022-01-10T12', '2022-01-10T18'],
       ['2022-01-11T00', '2022-01-11T06', '2022-01-11T12', ...,
        '2022-01-15T06', '2022-01-15T12', '2022-01-15T18'],
       ...,
       ['2022-12-17T00', '2022-12-17T06', '2022-12-17T12', ...,
        '2022-12-21T06', '2022-12-21T12', '2022-12-21T18'],
       ['2022-12-22T00', '2022-12-22T06', '2022-12-22T12', ...,
        '2022-12-26T06', '2022-12-26T12', '2022-12-26T18'],
       ['2022-12-27T00', '2022-12-27T06', '2022-12-27T12', ...,
        '2022-12-31T06', '2022-12-31T12', '2022-12-31T18']],
      dtype='datetime64[h]')

# Type Annotations and Documentation

Many of the functions are type annotated to include the expected shape and data type of arrays.
the `mesoscaler._typing` module contains supports compatiblity from Python 3.9-3.11.

In [13]:
from typing import Concatenate, Any, TypeAlias, Literal as L
from mesoscaler._typing import Array, Nd, N

assert Nd is Concatenate  # type: ignore


FloatArray: TypeAlias = Array[[...], np.float_]  # type: type[np.ndarray[Nd[...], np.dtype[np.floating[Any]]]]
ShapedArray: TypeAlias = Array[[N, N, L[4]], Any]

FloatArray, ShapedArray

(numpy.ndarray[(Ellipsis,), numpy.dtype[numpy.float64]],
 numpy.ndarray[(mesoscaler._typing.N, mesoscaler._typing.N, typing.Literal[4]), numpy.dtype[typing.Any]])